
# Programming Assignment: Рекомендательные системы

Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

##Задание

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

In [ ]:
Вы должны обратить внимание на несколько моментов:

В каждом из 4-х пунктов от вас требуется по 4 метрики, округленных до двух знаков.
Выкидывать выборку при обучении не нужно. Выкидывайте только, когда начинаете подсчет метрик.
Данные в каждом кортеже состоят из просмотров (до ";") и покупок (после ";").
В первом пункте вам необходимо пройтись во всем просмотрам всех сессий из train, подсчитать количество для каждого id,
отсортировать по количеству и подсчитать метрики используя тот же самый train.
Необходимо сохранять порядок сортировки между элементами с одинаковым значением. 
По умолчанию в pandas при вызове sort_values() используется quicksort, которая не гарантирует выполнение этого свойства. 
Если вы используете pandas, убедитесь, что вы используете стабильную сортировку, например mergesort.
Пример метрик, подсчитанный для кортежа:

Просмотры: ['59', '60', '61', '62', '60', '63', '64', '65', '66', '61', '67', '68', '67']
Покупки: ['67','60', '63']
Количество посчитанных значений по просмотрам по всем сессиям должно равняться: [('59', 1), ('60', 2), ('61', 2), ('62', 1), ('63', 6), ('64', 3), ('65', 2), ('66', 2), ('67', 2), ('68', 2)]
Отсортированный массив должен получиться: ['63', '64', '60', '61', '65', '66', '67', '68', '59', '62']
Для подсчета Precision@5 берем только первые 5 просмотров в отсортированном массиве, находим количество пересечений с покупками, и делим на 5. Должно получиться 0.4.
Для подсчета Recall@5 берем также первые 5 просмотров, находим количество пересечений с покупками и делим на количество покупок. Должно получиться 0.666.
Для подсчета Precision@1 берем первый просмотр и находим есть ли он в списке покупок. Делим на 1. Должно получиться 1.0
Для подсчета Recall@1 берем первый просмотр и находим есть ли он в списке покупок. Делим на количество покупок. Должно получиться 0.333.
 


In [ ]:
1.Построить частоты просмотров и покупок для каждого товара

2. Для вычисления точности и полноты по частоте просмотров или покупок для 1 id, мы:

сортируем просмотренные товары по частоте просмотров (или покупок);

удаляем дубликаты (назовем массив без дубликатов W), тут важно сохранить порядок просмотренных товаров, поэтому просто set(viewed) не стоит применять

берем первые k массива W;

удаляем дубликаты из списка купленных товаров

считаем мощность пересечения получившихся массивов (например, просмотры - [1, 2, 2, 3, 4, 5], покупки - [2, 5, 6, 10]) Тогда после удаления дубликатов получем пересечени [2,5], мощность (кол-во элементов) равна двум;

в случае точности делим полученный результат на k, в случае полноты делим на число покупок (уникальных) по данному id. В нашем случае при k=5 получаем точность = 0.4, полнота = 0.5

3. Для вычисления средних показателей, просто считаем наши статистики по всем элементам (не забывая выкидывать те id, по которым не было покупок), суммируем и результат делим на количество элементов.
 0 Upvote
 · 
Hide 1 Reply


In [1]:
import numpy as np
import pandas as pd
from itertools import chain
from collections import Counter

In [5]:
df = pd.read_csv("./coursera_sessions_train.txt", sep=";", header=None) #';' отделяет просмотры от покупок
df.columns = ["viewed", "bought"]

In [12]:
def getKey(item):
    return item[0]

In [13]:
sorted(df, key=getKey)

['bought', 'viewed']

In [ ]:
from collections import defaultdict, namedtuple

In [2]:
data, test_data = [], []

Record = namedtuple('Record', ['watched', 'bought'])
clr = lambda it: map(int, filter(None,it.split(',')))


NameError: name 'namedtuple' is not defined

In [ ]:
        
with open('coursera_sessions_train.txt', 'r') as f:
    for line in f.readlines():
        watched, bought = line.strip().split(';')
        data.append(Record(clr(watched), clr(bought)))

with open('coursera_sessions_test.txt', 'r') as f:
    for line in f.readlines():
        watched, bought = line.strip().split(';')
        test_data.append(Record(clr(watched)

In [57]:
df.head()

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [75]:
len(df.viewed)

50000

In [115]:
df.viewed.tolist()

[[0, 1, 2, 3, 4, 5],
 [9, 10, 11, 9, 11, 12, 9, 11],
 [16, 17, 18, 19, 20, 21],
 [24, 25, 26, 27, 24],
 [34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39],
 [42],
 [47, 48, 49],
 [59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67],
 [71, 72, 73, 74],
 [76, 77, 78],
 [84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86],
 [114, 77, 115, 116, 117, 118, 119, 120, 121, 120, 122, 123, 124],
 [129, 130, 131, 132, 133, 134],
 [93, 137, 138, 139, 140, 141, 142, 141, 143, 144],
 [97, 155, 156, 155, 157, 93],
 [163, 87, 129, 164, 129, 130, 165, 166],
 [173, 174, 175, 174, 176],
 [136, 192],
 [195, 196],
 [138, 198, 199, 127],
 [206, 207],
 [216, 217],
 [220],
 [223, 224, 225],
 [241, 242, 243, 244, 245, 246, 247, 248, 249, 242],
 [253, 254, 169],
 [262, 263, 262, 264],
 [265,
  266,
  267,
  268,
  269,
  270,
  269,
  268,
  271,
  272,
  273,
  274,
  275,
  276,
  277,
  269,
  278,
  279,
  280,
  269,
  281,
  282,
  283,
  269],
 [291],
 [296, 297, 298, 299],
 [303, 304, 305, 306, 307, 308, 309,

In [114]:

Counter(elem[0] for elem in df.viewed.tolist())

Counter({0: 1,
         9: 1,
         16: 4,
         24: 4,
         34: 3,
         42: 2,
         47: 1,
         59: 1,
         71: 21,
         76: 1,
         84: 2,
         114: 42,
         129: 2,
         93: 15,
         97: 1,
         163: 1,
         173: 3,
         136: 14,
         195: 5,
         138: 3,
         206: 2,
         216: 1,
         220: 1,
         223: 9,
         241: 28,
         253: 1,
         262: 85,
         265: 1,
         291: 6,
         296: 1,
         303: 1,
         332: 2,
         8: 46,
         352: 7,
         361: 27,
         161: 13,
         379: 1,
         384: 13,
         390: 15,
         406: 6,
         413: 1,
         423: 1,
         433: 1,
         439: 3,
         446: 2,
         458: 4,
         460: 1,
         475: 1,
         481: 1,
         486: 5,
         489: 5,
         496: 2,
         504: 1,
         510: 20,
         519: 17,
         522: 3,
         531: 1,
         547: 21,
         551: 36,

In [100]:
 df.dropna(axis=0, how='any') #Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.

,viewed,bought
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86
19,"138,198,199,127",199
30,"303,304,305,306,307,308,309,310,311,312",303
33,"352,353,352",352
55,519,519
64,"599,600,601,602","603,604,602,599,605,606,600"
72,"687,688,689,690,691,690,688,690,688,692","690,688"
89,"850,851,852",851
93,"879,884,170,137,170,879,884,879,885,886,879,88...",879


In [106]:
type (df.bought)

pandas.core.series.Series

In [112]:
list(map(df.viewed[7].split(',')))

AttributeError: 'list' object has no attribute 'split'

In [102]:

def handle_row(x):
    if x == x:
        return list(map(int, filter(None, x.split(','))))
    else:
        return np.nan

In [103]:
df["viewed"] = df.viewed.apply(handle_row)

AttributeError: 'list' object has no attribute 'split'

In [97]:
viewed=pd.unique(df['viewed'])
viewed[4]

'34,35,36,34,37,35,36,37,38,39,38,39'

In [ ]:
pd.unique(df[['Col1', 'Col2']].values.ravel())

In [87]:
for i in range (0,len(df.viewed)):
    if type (df.bought[i])!=str:
        continue
    else
    j=+1
    df.viewed=df.viewed[i].split(',')

SyntaxError: invalid syntax (<ipython-input-87-a0d5fbea237d>, line 4)

In [85]:
df.viewed

0                                              0,1,2,3,4,5
1                                     9,10,11,9,11,12,9,11
2                                        16,17,18,19,20,21
3                                           24,25,26,27,24
4                      34,35,36,34,37,35,36,37,38,39,38,39
5                                                       42
6                                                 47,48,49
7                   59,60,61,62,60,63,64,65,66,61,67,68,67
8                                              71,72,73,74
9                                                 76,77,78
10                     84,85,86,87,88,89,84,90,91,92,93,86
11       114,77,115,116,117,118,119,120,121,120,122,123...
12                                 129,130,131,132,133,134
13                  93,137,138,139,140,141,142,141,143,144
14                                   97,155,156,155,157,93
15                          163,87,129,164,129,130,165,166
16                                     173,174,175,174,1

In [68]:



from collections import OrderedDict

In [70]:
OrderedDict(sorted(df.viewed(), key=lambda t: t[0]))

TypeError: 'Series' object is not callable

In [66]:
sorted(df.viewed(), key=lambda t:t[0], reverse=True)

TypeError: 'Series' object is not callable

In [60]:
df.viewed[1]

'9,10,11,9,11,12,9,11'

In [ ]:
for i in range(0,len(df.viewed)):

In [ ]:
dictionary sorted by key
>>> OrderedDict(sorted(d.items(), key=lambda t: t[0]))

In [ ]:
per_amount = 0 # число пересечений покупок и просмотров в текущей сессии
suc = 0 # число просмотров, которое привело к покупкам
pre5 = 0
rec5 = 0
pre1 = 0
rec1 = 0
for i in range(0,len(train.index)):
    if type(train.buy[i]) != str:
        continue
    else:
        suc += 1
        train_see_s = train.see[i].split(',') # переводим строку просмотров в лист
        train_see_s_clear = pd.pd.unique(train_see_s) # удаляем повторы 
        # массив просмотров в сессии и частота покупок 
        train_see_count =[]
        for j in range(0, len(train_see_s_clear)):
            train_see_count.append((train_see_s_clear[j],bagsofwords_buy[train_see_s_clear[j]]))
        train_see_count = np.array([[int(train_see_count[m][k]) for k in range(0,2)]for m in range(0,len(train_see_count))])
        train_see_count = train_see_count[train_see_count[:,1].argsort()]
        train_see_count = np.flipud(train_see_count)
        #массив покупок
        train_buy = train.buy[i].split(',')
        train_buy = np.array([int(train_buy[k]) for k in range(0,len(train_buy))])
        # расчет метрик
        if len(train_see_count[:,0]) >=5:
            per_amount = len(list(set(train_buy) & set(train_see_count[0:4,0])))
            pre5 += float(per_amount)/5
            rec5+= float(per_amount)/len(train_buy)
        if len(train_see_count[:,0]) <5:
            per_amount = len(list(set(train_buy) & set(train_see_count[:,0])))
            pre5 += float(per_amount)/5
            rec5+= float(per_amount)/len(train_buy)
        pre1 += len(list(set(train_buy) & set(train_see_count[0:1,0])))
        rec1 += float(len(list(set(train_buy) & set(train_see_count[0:1,0]))))/len(train_buy) 
        

pre5 =(float(pre5))/suc
rec5 = rec5/suc
pre1 = float(pre1)/suc
rec1 = rec1/suc

In [39]:

def handle_row(x):
    if x == x:
        return list(map(int, filter(None, x.split(','))))
    else:
        return np.nan

In [40]:
df["watched"] = df.watched.apply(handle_row)

In [41]:
df["bought"] = df.bought.apply(handle_row)

In [58]:
df

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


In [43]:

cnt_watched = Counter(
     list(chain.from_iterable(
        df.watched.dropna().values
    ))
)

In [44]:

cnt_bought = Counter(
     list(chain.from_iterable(
        df.bought.dropna().values
    ))
)

In [45]:
def recommendation(arr, recommendor, k=1):
    rec = np.array(list(map(
            lambda x: recommendor[x], 
            arr
        )))
    k = min(k, len(rec))
    rec = rec.argsort()[-k:][::-1]
    return list(map(lambda x: arr[x], rec))

In [46]:

def prec_and_rec_at_k(watched, bought, recommendor, k):
    rec = recommendation(watched, recommendor, k)
    k = len(rec)
    right_pred = sum([recommend in rec for recommend in bought])
    return right_pred/k, right_pred/len(bought)

In [47]:
df.dropna(inplace=True)

In [48]:

def handle_k(k):
    df["precision_recall_watched"] = \
        df[["watched", "bought"]].apply(
            lambda x: prec_and_rec_at_k(x["watched"], x["bought"], 
                                        cnt_watched, 1),
            axis=1
        )
    df["precision_watched"] = df["precision_recall_watched"].apply(
        lambda x: x[0]
    )
    df["recall_watched"] = df["precision_recall_watched"].apply(
        lambda x: x[1]
    )
    return df["precision_watched"].mean(), df["recall_watched"].mean()

In [49]:
def results():
    res1, res2 = handle_k(1)
    res3, res4 = handle_k(5)
    return res1, res2, res3, res4

In [50]:
results()

(0.5171840354767184,
 0.4473807134907463,
 0.5171840354767184,
 0.4473807134907463)

In [55]:
 df["recall_1_lookat"]

KeyError: 'recall_1_lookat'

In [51]:
lookat_res = [
    df["recall_1_lookat"].mean(),
    df["precision_1_lookat"].mean(),
    df["recall_5_lookat"].mean(),
    df["precision_5_lookat"].mean(),
]

KeyError: 'recall_1_lookat'

In [52]:

buy_res = [
    df["recall_1_buy"].mean(),
    df["precision_1_buy"].mean(),
    df["recall_5_buy"].mean(),
    df["precision_5_buy"].mean(),
]

KeyError: 'recall_1_buy'

In [53]:
# Функция сохранения в файл ответа, представленного массивом
def save_answerArray(fname,array):
    with open(fname,"w") as fout:
        fout.write(" ".join([str(el) for el in array]))

In [54]:
results = list(map(lambda x: "{0:.2f}".format(x), lookat_res))

NameError: name 'lookat_res' is not defined